In [1]:
import json
import pathlib

from tabulate import tabulate

In [2]:
report_dir = "/home/mdafifal.mamun/notebooks/triagerX/training/reports"

In [3]:
d = "/home/mdafifal.mamun/notebooks/triagerX/triagerx/data/df_all_summarized.csv"
import pandas as pd

df = pd.read_csv(d)

df.head()

,Unnamed: 0,issue_number,issue_title,issue_body,issue_url,issue_state,creator,labels,assignees,component,summary
0,0,2,Build instructions link in the README.md point...,The `Build instructions` link in the `README.m...,https://github.com/eclipse-openj9/openj9/issues/2,closed,aarongraham9,NaN,gireeshpunathil,NaN,"The bug report highlights an issue with the ""B..."
1,1,3,FAQ link in the README is broken,FAQ link in the README leads to: http://www.ec...,https://github.com/eclipse-openj9/openj9/issues/3,closed,dorrab,NaN,mpirvu,NaN,The bug report states that the FAQ link in the...
2,2,5,Link to DockerFile on build instruction page i...,Link for DockerFile on [build instruction page...,https://github.com/eclipse-openj9/openj9/issues/5,closed,r30shah,NaN,r30shah,NaN,The bug report describes an issue with the lin...
3,3,11,HOWTO Request: Managing changes across depende...,"Like all projects, OpenJ9 builds on the should...",https://github.com/eclipse-openj9/openj9/issue...,open,mgaudet,question,hzongaro,NaN,Here is a summarized version of the bug report...
4,4,13,Compilation Output is too Verbose,The output when compiling the OpenJ9 source co...,https://github.com/eclipse-openj9/openj9/issue...,closed,rservant,"enhancement, comp:build",hzongaro,comp:build,Here is a summarized version of the bug report...


In [4]:
print(df.iloc[1270].summary)

Here is a summarized version of the bug report in natural language:
The bug report describes a failure in the cmdLineTester_dumpromclasstests_0 test on AIX. The test was run using the BCI agent core and the jdmpview command to analyze a core dump file. The test started at 2018/11/06 01:26:12 Central Standard Time and took 959 milliseconds to execute. The test result was FAILED. The output from the test indicates that the dump file could not be loaded and a valid ImageFactory could not be created. The test was expecting specific output matches, but none of them were found. The expected matches included "java/lang/Object", "intermediateClassData", "DDRInteractiveCommandException", "unable to read", "could not read", and "dump event".


In [5]:
print(df.iloc[1270].issue_url)

https://github.com/eclipse-openj9/openj9/issues/3562


In [10]:
def compute_metrics(report_prefix: str):
    accuracies = []
    top3_accuracies = []
    precisions = []
    recalls = []
    f1s = []
    
    reports = list(pathlib.Path(report_dir).glob(report_prefix))

    print(f"Report prefix: {report_prefix}, total reports:  {len(reports)}")
    
    for report_path in sorted(reports):
        with open(report_path.as_posix(), "r") as json_file:
            json_data = json.load(json_file)
            avg_type = "macro avg"

            accuracies.append(json_data["accuracy"])
            top3_accuracies.append(json_data["top3_accuracy"])
            precisions.append(json_data[avg_type]["precision"])
            recalls.append(json_data[avg_type]["recall"])
            f1s.append(json_data[avg_type]["f1-score"])

    avg_accuracy = sum(accuracies) / len(accuracies)
    avg_top3_accuracy = sum(top3_accuracies) / len(top3_accuracies)
    avg_precision = sum(precisions) / len(precisions)
    avg_recall = sum(recalls) / len(recalls)
    avg_f1_score = sum(f1s) / len(f1s)

    return {
        "average_accuracy": avg_accuracy,
        "average_top3_accuracy": avg_top3_accuracy,
        "average_test_precision": avg_precision,
        "average_test_recall": avg_recall,
        "average_test_f1_score": avg_f1_score
    }

## Results

In [13]:
training_type = "comp"

no_preprocessing = compute_metrics(f"classification_report_{training_type}_raw*")
sp_tokens = compute_metrics(f"classification_report_{training_type}_sp*")
summary_description = compute_metrics(f"classification_report_{training_type}_summary_description*")
summary_sp_tokens = compute_metrics(f"classification_report_{training_type}_summary_sp_tokens*")

data = [
    ['No Preprocessing', no_preprocessing['average_accuracy'], no_preprocessing['average_top3_accuracy'], no_preprocessing['average_test_precision'], no_preprocessing['average_test_recall'], no_preprocessing['average_test_f1_score']],
    ['Special Tokens (sp_tokens)', sp_tokens['average_accuracy'], sp_tokens['average_top3_accuracy'], sp_tokens['average_test_precision'], sp_tokens['average_test_recall'], sp_tokens['average_test_f1_score']],
    ['Summary + Description', summary_description['average_accuracy'], summary_description['average_top3_accuracy'], summary_description['average_test_precision'], summary_description['average_test_recall'], summary_description['average_test_f1_score']],
    ['Summary + Special Tokens', summary_sp_tokens['average_accuracy'], summary_sp_tokens['average_top3_accuracy'], summary_sp_tokens['average_test_precision'], summary_sp_tokens['average_test_recall'], summary_sp_tokens['average_test_f1_score']]
]

# Define the headers
headers = ['Preprocessing Method', 'Average Accuracy', 'Average Top-3 Accuracy', 'Average Test Precision', 'Average Test Recall', 'Average Test F1 Score']

print(tabulate(data, headers=headers, tablefmt='pretty'))

Report prefix: classification_report_comp_raw*, total reports:  3
Report prefix: classification_report_comp_sp*, total reports:  3
Report prefix: classification_report_comp_summary_description*, total reports:  3
Report prefix: classification_report_comp_summary_sp_tokens*, total reports:  3
+----------------------------+--------------------+------------------------+------------------------+---------------------+-----------------------+
|    Preprocessing Method    |  Average Accuracy  | Average Top-3 Accuracy | Average Test Precision | Average Test Recall | Average Test F1 Score |
+----------------------------+--------------------+------------------------+------------------------+---------------------+-----------------------+
|      No Preprocessing      | 0.7529665587918015 |   0.9848975188781014   |   0.6285117814438443   | 0.6600723955075787  |  0.6260089095834536   |
| Special Tokens (sp_tokens) | 0.7664509169363538 |   0.9848975188781014   |   0.6229234484485358   | 0.71572822220